<a href="https://colab.research.google.com/github/LuneRemer/Korean-Comments-Keyword-Extraction/blob/main/Keyword_Extraction_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests_html

     |████████████████████████████████| 83 kB 2.7 MB/s 
     |████████████████████████████████| 138 kB 61.6 MB/s 
     |████████████████████████████████| 103 kB 50.0 MB/s 
     |████████████████████████████████| 127 kB 62.2 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=f059e14368bf4573ff1159751da81ebabeafc19b9b7a787ce86d47f16179067f
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=9e2102231273bf98f78eab28af6a896a065af10b2560c491c41ddf86b00adf80
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built fake-useragent parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not

In [2]:
!pip install customized_konlpy

     |████████████████████████████████| 881 kB 34.0 MB/s 
     |████████████████████████████████| 448 kB 47.9 MB/s 
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
cd drive/MyDrive/colab/BERT\ Keyword\ Extraction

/content/drive/MyDrive/colab/BERT Keyword Extraction


In [4]:
import json
from requests_html import HTMLSession
from bs4 import BeautifulSoup

from urllib.request import urlopen, Request

import re
from gensim.models import Word2Vec
# from ckonlpy.tag import Okt
from ckonlpy.tag import Twitter
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd
from fake_useragent import UserAgent
from konlpy.tag import Hannanum

In [5]:
texts_url = []
summary_url = []

In [6]:
def crawling_url(url):
    global texts_url
    global page_url
    session = HTMLSession()
    res = session.get(url)

    soup = BeautifulSoup(res.content, 'html.parser')
#     print(soup)
    urls = soup.find("div", class_="rankingnews_box_wrap _popularRanking").find_all("a", class_="list_title nclicks('RBP.cmtnws')")    
#     print(urls)
    print(len(urls))
    for x in range(len(urls)):        
        check = urls[x]['href']
#         print(check)
        texts_url.append(check)
        if 'mid=shm' in check:
            if 'oid=' in check:
                if '/main/ranking/read.naver' in check:
                    if 'aid=' in check:
                        if 'rankingType=RANKING' in check:
                            texts_url.append(check)  

#* 뉴스기사의 제목을 크롤링
def crawling_title(url):
    session = HTMLSession()
    res = session.get(url)

    soup = BeautifulSoup(res.content, 'html.parser')
    article = soup.find('div', attrs={'id': 'articleBodyContents'})
    title = soup.find('h3', attrs={'id': 'articleTitle'})
    if title:
        return title.text
    else:
        return "No such tag, title"

#* 뉴스기사의 요약문을 크롤링
def crawling_brief_article(url):
    session = HTMLSession()
    res = session.get(url)

    json_summary = res.text
    json_summary = json_summary.split("summary\":\"")[1].split("\"}")[0]
    json_summary = json_summary.replace("<br/>","")

    return json_summary

#* 뉴스 댓글 api의 resoponse를 json 형식으로 바꿈.
def getJson_NewsInfo(resp):
    x = 0
    while(resp[x] != '{'):
        x += 1
     # print(response[x:])

    z = len(resp)-1
    while(resp[z] != '}'):
        z -= 1
    # print(response[x:y]+'}')

    rank_json = json.loads(resp[x:z]+'}')

    return rank_json
  
#* 뉴스의 url에서 oid와 aid를 수집
def getOid_Aid(uri):
    global oid
    global aid
    oid.append(uri.split("oid=")[1].split("&")[0])
    aid.append(uri.split("aid=")[1].split("&ranking")[0])

In [7]:
hannanum = Hannanum()

In [8]:
crawling_url("https://news.naver.com/main/ranking/popularMemo.naver")


texts_url = set(texts_url)
texts_url = list(texts_url)

# print("*********url 정보***********")
print(texts_url)
print(len(texts_url))


oid = []
aid = []
headers = []
com_url = []

60
['/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=421&aid=0005642966&rankingType=RANKING', '/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=214&aid=0001152606&rankingType=RANKING', '/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=437&aid=0000277857&rankingType=RANKING', '/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=016&aid=0001896699&rankingType=RANKING', '/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=052&aid=0001649727&rankingType=RANKING', '/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=437&aid=0000277859&rankingType=RANKING', '/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=214&aid=0001152636&rankingType=RANKING', '/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=023&aid=0003645069&rankingType=RANKING', '/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=028&aid=0002563048&rankingType=RANKING', '/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=031&aid=0000627669&rankingType=RANKING', '/main/ranking/read.naver?

In [9]:
# p = re.compile("[0-9]+")
    # text = p.sub(" ", text)

def pre_processing(title):
  title = title.replace("\"",'')
  title = title.replace("…"," ")
  title = title.replace("\'",'')
  title = title.replace("“",'')
  title = title.replace("·",'')  
  title = title.replace(",",'')  
  title = re.sub('([ㄱ-ㅎㅏ-ㅣ]+)', '', title)
  title = re.sub('[^a-zA-Z0-9ㄱ-ㅣ가-힣%.]',' ',title)

  return title


for i in range(len(texts_url)):
    texts_url[i] = 'https://news.naver.com' + texts_url[i]

In [10]:
titles = []
for i in tqdm(range(len(texts_url))):
  titles.append(crawling_title(texts_url[i]))

title_array = []
for title in titles:
  title = pre_processing(title)  
  temp = title.split(' ')
  title_array.append(temp)

100%|██████████| 60/60 [00:28<00:00,  2.08it/s]


In [11]:
twitter = Twitter()
for i in range(len(title_array)):  
    twitter.add_dictionary(title_array[i], 'Noun')
twitter.add_dictionary('사회주의', 'Noun', force=False)
twitter.add_dictionary('공산주의', 'Noun', force=False)
twitter.add_dictionary('돈벌이', 'Noun', force=False)
twitter.add_dictionary('이죄명', 'Noun', force=False)

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [12]:
twitter.add_dictionary('대통령', 'Noun', force=False)

In [13]:
def han_tokenization(text):
  global twitter
  global hannanum

  #* 명사로 인식하지만, 키워드로써 맞지 않는 단어들
  #* 여러 댓글들에서 공통으로 나오는 단어들
  text = text.replace("한다",'')
  text = text.replace("보고",'')
  text = text.replace("정말",'')
  text = text.replace("구나",'')
  text = text.replace("맞고",'')
  text = text.replace("아직",'')
  text = text.replace("저런",'')
  text = text.replace("까지",'')
  text = text.replace("닮고",'')
  text = text.replace("하나",'')
  text = text.replace("그냥",'')
  text = text.replace("면서",'')
  text = text.replace("하다",'')
  text = text.replace("부리고",'')
  text = text.replace("진짜",'')
  text = text.replace("더니",'')
  text = text.replace("던데",'')
  text = text.replace("있다",'')
  text = text.replace("없다",'')
  text = text.replace("만도",'')
  text = text.replace("그것",'')
  text = text.replace("이것",'')
  text = text.replace("이건",'')
  text = text.replace("하는",'')
  text = text.replace("제발",'')
  text = text.replace("이번",'')
  text = text.replace("그래서",'')
  text = text.replace("주변",'')
  text = text.replace("이지",'')
  text = text.replace("팔이",'')
  text = text.replace("한번",'')
  text = text.replace("해먹",'')
  text = text.replace("속이",'')
  text = text.replace("간다",'')
  text = text.replace("이거",'')
  text = text.replace("다른",'')
  text = text.replace("그만",'')
  text = text.replace("하자",'')
  text = text.replace("해라",'')
  text = text.replace("지금",'')
  text = text.replace("때문",'')
  text = text.replace("만큼",'')
  text = text.replace("이면",'')
  


  tag_array = twitter.pos(text)
  temp_tag = []
  temp_real_tag = []
  for i in range(len(tag_array)):
    if tag_array[i][1] == 'Noun' or tag_array[i][1] == 'Alpha':
      temp_tag.append(tag_array[i][0])
  for i in range(len(temp_tag)):
    if hannanum.pos(temp_tag[i])[0][1]=='N' or hannanum.pos(temp_tag[i])[0][1]=='F':
      temp_real_tag.append(temp_tag[i])
  tagged_text = ' '.join(temp_real_tag)
  return tagged_text

In [14]:
for y in range(len(texts_url)):
    a = texts_url[y]
    getOid_Aid(a)

#* https://tts.news.naver.com/article/052/0001647706/summary
for i in range(len(texts_url)):
    summ_url = "https://tts.news.naver.com/article/{}/{}/summary".format(oid[i], aid[i])
    summary_url.append(summ_url)


for x in range(len(texts_url)):
    comps_url = []
    for y in range(1, 30):
        comps_url.append("https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_politics_m1&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" +
                         oid[x] + "%2C" + aid[x] + "&categoryId=&pageSize=100&indexSize=100&groupId=&listType=OBJECT&pageType=more&page=" + str(y) + "&refresh=false&sort=FAVORITE")
    # print(comps_url)
    com_url.append(comps_url)
# print(com_url[0][1])

useragent = UserAgent()

for v in range(len(texts_url)):
    headers.append({
        'referer': texts_url[v],
        'User-Agent': useragent.chrome
    })

print(len(texts_url))
print(len(summary_url))

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


60
60


In [17]:
from sklearn.metrics import silhouette_score

def get_kmeans_k(X):
  k_range = range(2,6)

  best_n = -1
  best_silhouette_score = -1

  for k in k_range:
    try:
      kmeans = KMeans(n_clusters=k)
      kmeans.fit(X)
      clusters = kmeans.predict(X)

      score = silhouette_score(X, clusters)
      
      if score > best_silhouette_score:
        best_n = k
        best_silhouette_score = score
    except:
      continue

  return best_n

In [19]:
contents_array = []
num_comments = 0
# keyword = pd.DataFrame(columns=['news_title','keyword_candidate'])
df_list = []

for v in tqdm(range(len(texts_url))):    
    num_comments = 0  
    # print('ha')
    graph_exists = False
    contents_array = []
    for y in range(len(com_url[v])):
        
        resp = urlopen(
            Request(com_url[v][y], headers=headers[v])).read().decode('utf-8')
        
        if not '"commentList":[]' in resp:
            rank_json = getJson_NewsInfo(resp)
            # print(rank_json)

            if rank_json['result']['commentList']:
                num_comments += len(rank_json['result']['commentList'])
                for w in range(len(rank_json['result']['commentList'])):                    
                    contents = str(rank_json['result']['commentList'][w]['contents'])
                    contents = pre_processing(contents)
                    contents_array.append(contents)
        else:
            break
    
    for i in range(len(contents_array)):
        contents_array[i] = han_tokenization(contents_array[i])
    comments_token = []
    for i in range(len(contents_array)):
        temp = contents_array[i].split(' ')
        comments_token.append(temp)

    try:
        word2vec_model = Word2Vec(comments_token, window=4, min_count=20)
    except:
        continue
    print("Done word2vec Training")

    model_word_vector = {}
    model_vector_word = {}
    for word in word2vec_model.wv.vocab:
        model_word_vector[word] = word2vec_model[word]
        # print(model_word_vector[word])
        model_vector_word[str(word2vec_model[word])] = word

    

    X = word2vec_model[word2vec_model.wv.vocab]

    n = get_kmeans_k(X)

    if n == -1:
      continue

    kmeans = KMeans(n_clusters = n)
    kmeans.fit(X)

    kmeans_label = kmeans.labels_

    word_df = pd.DataFrame(columns=['word', 'cluster'])

    word_df['word'] = word2vec_model.wv.vocab
    # word_df['cluster'] = dbscan_labels
    word_df['cluster'] = kmeans_label

    #* group by로 각 클러스터에서 1~2개씩 뽑음 

    for i in range(n):
      word_cluster_i = word_df.loc[word_df['cluster']==i]

      keyword_candidate_count = []
      for keyword_candidate in word_cluster_i["word"]:
          word_count = 0        
          for i in range(len(comments_token)):
              for word in comments_token[i]:
                  if word in keyword_candidate:
                      word_count += 1
                  elif keyword_candidate in word:
                      word_count += 1
          keyword_candidate_count.append(word_count)
      word_cluster_i['count'] = keyword_candidate_count
      df_i = word_cluster_i.sort_values(by=['count'], ascending=False, ignore_index=True)
      length_word_i = []
      for j in range(len(df_i)):
          length_word_i.append(len(df_i["word"][j]))
      df_i['length'] = length_word_i
      df_i = df_i.loc[(df_i["length"]>1) & (df_i["word"]!="하고") & (df_i["word"]!="이나") & (df_i["word"]!="이제") & (df_i["word"]!="려고")]
      df_i.reset_index(drop=True, inplace=True)

      for i in range(0,2):
          try:
            temp = []
            temp.append(titles[v])
            temp.append(df_i["word"][i])
            # temp.append(word2vec_model.most_similar_cosmul(df_i["word"][i])[0][0])
            df_list.append(temp)
          except:
            continue
      print(titles[v])
      print(df_i[0:2])
      # print(word2vec_model.most_similar_cosmul(df_i["word"][i])[0:2][0])

  0%|          | 0/60 [00:00<?, ?it/s]

Done word2vec Training


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'대장동 딛고'…이재명 44% vs 윤석열 33%, 이재명 40% vs 홍준표 37%
   word  cluster  count  length
0  여론조사        0    930       4
1   이재명        0    849       3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  2%|▏         | 1/60 [00:35<34:41, 35.28s/it]

...

Done word2vec Training
이준석 “이재명, 무능이냐 부패냐 딜레마”…野 ‘대장동 특검’ 천막농성
  word  cluster  count  length
0   국민        0     41       2
이준석 “이재명, 무능이냐 부패냐 딜레마”…野 ‘대장동 특검’ 천막농성
  word  cluster  count  length
0   특검        1     39       2
이준석 “이재명, 무능이냐 부패냐 딜레마”…野 ‘대장동 특검’ 천막농성
  word  cluster  count  length
0   국짐        2     36       2
1  이재명        2     35       3
Done word2vec Training


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWar

40대 가장 폭행한 만취녀 또 사과문자…"3000만원 드릴까 한다"
  word  cluster  count  length
0  합의금        0    254       3
1   정도        0    230       2
40대 가장 폭행한 만취녀 또 사과문자…"3000만원 드릴까 한다"
  word  cluster  count  length
0   합의        1    250       2
1   사과        1    185       2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWar

Done word2vec Training
홍준표·윤석열·유승민·원희룡, 野 대선 최종 경선 진출(2보)
  word  cluster  count  length
0  원희룡        0     32       3
홍준표·윤석열·유승민·원희룡, 野 대선 최종 경선 진출(2보)
  word  cluster  count  length
0  유승민        1     68       3
1  윤석열        1     41       3
홍준표·윤석열·유승민·원희룡, 野 대선 최종 경선 진출(2보)
  word  cluster  count  length
0  홍준표        2     63       3
홍준표·윤석열·유승민·원희룡, 野 대선 최종 경선 진출(2보)
  word  cluster  count  length
0  최재형        3     32       3


In [20]:
keyword_extract_df = pd.DataFrame(df_list)
keyword_extract_df.to_csv("data/keyword_silhouette_2.csv", index=False, encoding='utf-8-sig')